# Introduction
This notebook is a first part of [eargasm-spotify](https://github.com/adamsiemaszkiewicz/eargasm-music) repository meant to visualize data from the [eargasm music](https://open.spotify.com/user/eargasmusic?si=HtTLbkG6QoqkdKU3uTRjAQ) Spotify channel. 

# Set up environment

## Import libraries and functions

### System

In [1]:
import timeit

### Notebook

In [2]:
from tqdm.notebook import tqdm # for progress bars

### Data science

In [3]:
import pandas as pd # for data manipulation & analysis
import numpy as np # for linear algebra

# Load data
Let's load the data fetched from the [Eargasm Music](https://open.spotify.com/user/eargasmusic?si=cz4VzEi1RgaUup_g8-UnEg) Spotify channel using `eargasm_retrieve-data.ipynb`. The data set consists of the tracks with their features as well as a playlist name label.

In [4]:
audio_feature_df = pd.read_csv('audio_features.csv', index_col=0)

FileNotFoundError: ignored

In [5]:
!ls

sample_data


- https://github.com/tgel0/spotify-data
- https://www.kaggle.com/aeryan/spotify-music-analysis
- https://www.kaggle.com/arpita28/analysis-of-spotify-trends
- https://www.kaggle.com/jsongunsw/spotify-datasets
- https://www.kaggle.com/mohitkr05/spotify-data-visualization
- https://www.kaggle.com/karthiknc/spotify-visualisation-analysis
- https://www.kaggle.com/souhardyaganguly/spotify-svm
- https://www.kaggle.com/lowkimhoe/prediction-model-on-spotify-classification
- https://www.kaggle.com/pavansanagapati/birds-sounds-eda-spotify-urban-sound-eda
- https://www.kaggle.com/shadey/spotify
- https://medium.com/swlh/creating-waveforms-out-of-spotify-tracks-b22030dd442b
- https://vsupalov.com/analyze-spotify-music-library-with-jupyter-pandas/
- https://www.youtube.com/watch?v=v-9Mpe7NhkM
- http://harpolea.github.io/2018-05-09-spotipy
- https://medium.com/analytics-vidhya/music-genre-classification-with-python-51bff77adfd6
recommendations(seed_artists=None, seed_genres=None, seed_tracks=None, limit=20, country=None, **kwargs)